In [79]:
#Simulando erro obs: se tiver encoding utf-8 padrao o pandas ler sem prescisar passar parametro encoding
import pandas as pd

#pd.set_option('display.max_columns', None)# ver todas as colunas  
caminho_do_arquivo = r"C:\Users\TIAGO\Documents\curso_py&pg\V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig') 

In [80]:
df.head(3)

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrência,Data_da_Ocorrencia,Hora_da_Ocorrência,Municipio,UF,Regiao,Descricao_do_Tipo,...,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
0,5849,0853/2014,MARCIO AGUIAR DA SILVA,Incidente Grave,2014-05-13,16:50:00,SINOP,MT,Centro-Oeste,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,...,0.0,0.0,0.0,210N,L1P,C210,1724.0,6.0,CESSNA AIRCRAFT,verdadeiro
1,5849,0853/2014,MARCIO AGUIAR DA SILVA,Incidente Grave,2014-05-13,16:50:00,SINOP,MT,Centro-Oeste,EXCURSÃO DE PISTA,...,0.0,0.0,0.0,210N,L1P,C210,1724.0,6.0,CESSNA AIRCRAFT,verdadeiro
2,5845,0855/2014,AERO AGRICOLA SANTOS DUMONT LTDA,Acidente,2014-05-10,20:25:00,CACHOEIRA DO SUL,RS,Sul,PERDA DE CONTROLE EM VOO,...,0.0,0.0,0.0,140A,L1P,C140,680.0,2.0,CESSNA AIRCRAFT,verdadeiro


In [81]:
# Estamos filtrando nosso banco de dados para que seja puxado apenas estas colunas 
colunas =['Numero_da_Ocorrencia', 'Classificacao_da_Ocorrência', 'Data_da_Ocorrencia','Municipio', 'UF', 'Regiao','Nome_do_Fabricante','Modelo']
# fizemos uma tratativa chamando o datafreme e  passando para ele as colunas que queremos que seja mostrada 
df = df[colunas]
# Estamos renomeanddo umas das tabelas, retirando o seu acento para que não tenha erro futuros em nosso banco de dados 
df.rename(columns={'Classificacao_da_Ocorrência': 'Classificacao_da_Ocorrencia'}, inplace=True)

In [82]:
import psycopg2

dbname='python'
user='postgres'
password='root'
host = 'localhost'
port = '5432'


conexao = psycopg2.connect(dbname=dbname,user=user,password= password,host = host,port = port)
cursor = conexao.cursor()

# deletar os dados antes da carga
cursor.execute("delete from public.anac_mapeamento")

# carga de dados 
for indice,coluna_df in df.iterrows():
    cursor.execute(
    """
    INSERT INTO anac_mapeamento (
        Id, 
        Classificacao_da_Ocorrencia, 
        Dt_Ocorrencia,
        municipio, 	
        uf, 
        regiao,
        fabricante,
        Modelo
    ) VALUES (%s, %s, %s, %s, %s, %s, %s,%s) 
    """,
    (
        coluna_df["Numero_da_Ocorrencia"], # Valores que serão passados para sua respectiva colunas, onde para inserit sera lida em linha 
        coluna_df["Classificacao_da_Ocorrencia"],
        coluna_df["Data_da_Ocorrencia"],
        coluna_df["Municipio"],
        coluna_df["UF"],
        coluna_df["Regiao"],
        coluna_df["Nome_do_Fabricante"],
        coluna_df["Modelo"]
        
    )
    )

    
conexao.commit()
cursor.close()
conexao.close()